In [105]:
# -*- coding: utf-8 -*-
"""
The script demonstrates a comparison of vehicle times from AVL data to the
scheduled arrival time at one stop for one example trip.
"""

import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  

# these two modules are homemade
import gtfs
import arrivals
import time
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')


In [92]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('gtfs/')
stops = gtfs.load_stops('gtfs/')
trip_shapes = gtfs.load_trip_shapes('gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('gtfs/')
print 'Finished loading GTFS data.'

"""
# get the sample of parsed A	VL data.  Beware, takes a few minutes.
bustime = pd.read_csv('newdata_parsed.csv')#,parse_dates=dt_columns)
qstr = ('Trip == "MTA NYCT_MV_B6-Weekday-SDon-038500_M5_203" or '
    'Trip == "MTA NYCT_MV_B6-Weekday-SDon-036500_M5_202" or '
    'Trip == "MTA NYCT_MV_B6-Weekday-SDon-040000_M5_204"')
bustime = bustime.query(qstr)
bustime.drop_duplicates(['vehicleID','RecordedAtTime'],inplace=True)
bustime.set_index(['Line','Trip','TripDate','vehicleID','RecordedAtTime'],
                  inplace=True,drop=True,verify_integrity=True)

# for now, use a truncated data set.  just get data for one line (M5).
tripDateLookup = "2016-06-13"
lineLookup = "MTA NYCT_M5"
bustime_short = bustime.xs((lineLookup,tripDateLookup),level=(0,2))
del bustime # to free up memory
print 'Finished loading BusTime data and and slicing three trips of M5 line.'
"""

Finished loading GTFS data.


'\n# get the sample of parsed A\tVL data.  Beware, takes a few minutes.\nbustime = pd.read_csv(\'newdata_parsed.csv\')#,parse_dates=dt_columns)\nqstr = (\'Trip == "MTA NYCT_MV_B6-Weekday-SDon-038500_M5_203" or \'\n    \'Trip == "MTA NYCT_MV_B6-Weekday-SDon-036500_M5_202" or \'\n    \'Trip == "MTA NYCT_MV_B6-Weekday-SDon-040000_M5_204"\')\nbustime = bustime.query(qstr)\nbustime.drop_duplicates([\'vehicleID\',\'RecordedAtTime\'],inplace=True)\nbustime.set_index([\'Line\',\'Trip\',\'TripDate\',\'vehicleID\',\'RecordedAtTime\'],\n                  inplace=True,drop=True,verify_integrity=True)\n\n# for now, use a truncated data set.  just get data for one line (M5).\ntripDateLookup = "2016-06-13"\nlineLookup = "MTA NYCT_M5"\nbustime_short = bustime.xs((lineLookup,tripDateLookup),level=(0,2))\ndel bustime # to free up memory\nprint \'Finished loading BusTime data and and slicing three trips of M5 line.\'\n'

In [3]:
"""
# This function returns some elements of interest for each trip and stop
def trip_summary(avl_subset,t):    
    fail =  0
    trip_id = t[9:]
    stop_list = list(stop_times.loc[trip_id].index)
    # collect AVL data around each stop into a dict
    stop_pings = {}
    for stop_id in stop_list:
        try:
            stop_pings[stop_id] = arrivals.nearby_pings(stop_id,trip_id,stop_times,
                                                        stops,avl_subset)
        except:
            fail += 1
            continue    
    print str(time.time()) + ' - finished stop pings dict.'
    # make a summary table about the AVL data near each stop on a trip
    summary_columns = ['N','arrival_actual_estimated']
    summary_df = pd.DataFrame(columns=summary_columns,index=stop_pings.keys())
    for k, v in stop_pings.iteritems():
        # we want to know how many AVL pings were returned  nearby   
        newrow = {'N':len(v)}
        min_stamp = v['ResponseTimeStamp'].min()
        newrow['arrival_actual_estimated'] = min_stamp
        summary_df.loc[k] = newrow
    print str(time.time()) + ' - finished ' + t[9:]
    return summary_df

# now collect a dataframe with the summary data about all trips in the subset
day_summary = pd.DataFrame()
for t in bustime_short.index.get_level_values(0).unique():
    try:
        avl_subset = bustime_short.loc[t]
        summary_df = trip_summary(avl_subset,t)
        summary_df['trip_id'] = t[9:]
        summary_df['TripDate'] = tripDateLookup
        summary_df.reset_index(inplace=True)
        summary_df.rename(columns={'index':'stop_id'},inplace=True)
        day_summary = day_summary.append(summary_df)
    except:
        print t
day_summary.set_index(['trip_id','stop_id'],inplace=True)
day_summary = day_summary.join(stop_times['arrival_time'])

print day_summary.shape
"""


"\n# This function returns some elements of interest for each trip and stop\ndef trip_summary(avl_subset,t):    \n    fail =  0\n    trip_id = t[9:]\n    stop_list = list(stop_times.loc[trip_id].index)\n    # collect AVL data around each stop into a dict\n    stop_pings = {}\n    for stop_id in stop_list:\n        try:\n            stop_pings[stop_id] = arrivals.nearby_pings(stop_id,trip_id,stop_times,\n                                                        stops,avl_subset)\n        except:\n            fail += 1\n            continue    \n    print str(time.time()) + ' - finished stop pings dict.'\n    # make a summary table about the AVL data near each stop on a trip\n    summary_columns = ['N','arrival_actual_estimated']\n    summary_df = pd.DataFrame(columns=summary_columns,index=stop_pings.keys())\n    for k, v in stop_pings.iteritems():\n        # we want to know how many AVL pings were returned  nearby   \n        newrow = {'N':len(v)}\n        min_stamp = v['ResponseTimeStamp

In [46]:
# day_summary.to_csv('day_summary.csv')

data = pd.read_csv('longjsons_parsed.csv')

qstr = ('Trip == "MTA NYCT_FB_B6-Weekday-SDon-043400_B41_16"')
bustime = data.query(qstr)
bustime.drop_duplicates(['vehicleID','RecordedAtTime'],inplace=True)
bustime.set_index(['Line','Trip','TripDate','vehicleID','RecordedAtTime'],
                  inplace=True,drop=True,verify_integrity=True)
# for now, use a truncated data set.  just get data for one line (M5).
tripDateLookup = "2016-05-25"
lineLookup = "MTA NYCT_B41"
avl_long = bustime.xs((lineLookup,tripDateLookup),level=(0,2))
del bustime,data # to free up memory



In [13]:
from scipy import spatial

In [83]:
def time_at_location(lat,lon,trip_id,avl_data,stop_times,radius=0.001):
        trip_stop_times = stop_times.loc[trip_id]
        lookup = 'MTA NYCT_' + trip_id # lookup string used to query AVL data
        avl_subset = avl_data.query('Trip == @lookup')
        #print avl_subset
        points = zip(avl_subset.Longitude.ravel(), avl_subset.Latitude.ravel())
        tree = spatial.KDTree(points) # implement KDTree class
        lookup_point = [lon,lat]
        nearby_points = tree.query_ball_point(lookup_point, radius)
        #print nearby_points
        locations = avl_subset.iloc[nearby_points][['Latitude','Longitude']]
        times = avl_subset.iloc[nearby_points].index.get_level_values('RecordedAtTime')
        df = pd.DataFrame(avl_subset.iloc[nearby_points][['Latitude','Longitude']])
        df['RecordedAtTime'] = times
        df = df.set_index(['RecordedAtTime'])
        #resampled = df.resample('S').interpolate()
        return df#resampled
avl_data = avl_long
radius = 0.01
trip_id = "FB_B6-Weekday-SDon-043400_B41_16"
lat, lon = 40.6763783,-73.983488



time_at_location(lat,lon,trip_id,avl_data,stop_times,radius)    

,Latitude,Longitude
RecordedAtTime,,
2016-05-25T07:26:40.000-04:00,40.682170,-73.976072
2016-05-25T07:27:11.000-04:00,40.681098,-73.975304
2016-05-25T07:27:42.000-04:00,40.681126,-73.975324
2016-05-25T07:28:14.000-04:00,40.680564,-73.974913


In [86]:
stops.head()

,location_type,parent_station,stop_desc,stop_lat,stop_lon,stop_name,stop_url,zone_id
stop_id,,,,,,,,
100014,0,NaN,NaN,40.872562,-73.888153,BEDFORD PK BL/GRAND CONCOURSE,NaN,NaN
100017,0,NaN,NaN,40.876808,-73.889656,PAUL AV/W 205 ST,NaN,NaN
100018,0,NaN,NaN,40.880341,-73.886063,PAUL AV/W MOSHOLU PY S,NaN,NaN
100019,0,NaN,NaN,40.813496,-73.929489,GRAND CONCOURSE/E 138 ST,NaN,NaN
100020,0,NaN,NaN,40.816803,-73.927956,GRAND CONCOURSE/E 144 ST,NaN,NaN


In [87]:
stop_times.head()

arrival_time departure_time  \
trip_id                     stop_id                               
GH_B6-Weekday-004000_BX12_1 103794      00:40:00       00:40:00   
                            103795      00:40:22       00:40:22   
                            101739      00:42:26       00:42:26   
                            103255      00:44:00       00:44:00   
                            103945      00:49:00       00:49:00   

                                     stop_sequence  pickup_type  drop_off_type  
trip_id                     stop_id                                             
GH_B6-Weekday-004000_BX12_1 103794               1            0              0  
                            103795               2            0              0  
                            101739               3            0              0  
                            103255               4            0              0  
                            103945               5            0              0

In [90]:
trips.head()

,block_id,direction_id,route_id,service_id,shape_id,trip_headsign
trip_id,,,,,,
GH_B6-Weekday-004000_BX12_1,NaN,1,BX12,GH_B6-Weekday,BX120805,INWOOD BWY-207 ST- PEL PKY via FRDHAM RD
GH_B6-Weekday-009000_BX12_1,NaN,0,BX12,GH_B6-Weekday,BX120809,EDSON AV via FORDHAM RD via PELHAM PKY
GH_B6-Weekday-013000_BX12_1,NaN,1,BX12,GH_B6-Weekday,BX120792,INWOOD BWY-207 ST- PEL PKY via FRDHAM RD
GH_B6-Weekday-017000_BX12_1,NaN,0,BX12,GH_B6-Weekday,BX120809,EDSON AV via FORDHAM RD via PELHAM PKY
GH_B6-Weekday-021000_BX12_1,NaN,1,BX12,GH_B6-Weekday,BX120792,INWOOD BWY-207 ST- PEL PKY via FRDHAM RD


In [125]:
#trip_shapes.col
trip_shapes[trip_shapes.index=='BX010027']

,shape_pt_lat,shape_pt_lon,shape_pt_sequence
shape_id,,,
BX010027,40.809663,-73.92824,10001


In [112]:
trip_shapes.head()

,shape_pt_lat,shape_pt_lon,shape_pt_sequence
shape_id,,,
BX010026,40.809663,-73.928240,10001
BX010027,40.809663,-73.928240,10001
BX010028,40.881224,-73.909390,10001
BX010029,40.880626,-73.886019,10001
BX010030,40.880626,-73.886019,10001


In [ ]:
avl_data = avl_long
radius = 0.01
trip_id = "FB_B6-Weekday-SDon-043400_B41_16"
lat, lon = 40.6763783,-73.983488

trip_stop_times = stop_times.loc[trip_id]
lookup = 'MTA NYCT_' + trip_id # lookup string used to query AVL data
avl_subset = avl_data.query('Trip == @lookup')
#print avl_subset
points = zip(avl_subset.Longitude.ravel(), avl_subset.Latitude.ravel())
tree = spatial.KDTree(points) # implement KDTree class
lookup_point = [lon,lat]
nearby_points = tree.query_ball_point(lookup_point, radius)
#print nearby_points
locations = avl_subset.iloc[nearby_points][['Latitude','Longitude']]
times = avl_subset.iloc[nearby_points].index.get_level_values('RecordedAtTime')
df = pd.DataFrame(avl_subset.iloc[nearby_points][['Latitude','Longitude']])
df['RecordedAtTime'] = times
df = df.set_index(['RecordedAtTime'])
#resampled = df.resample('S').interpolate()
return df#resampled

time_at_location(lat,lon,trip_id,avl_data,stop_times,radius) 

This will be one example of one route

In [127]:
stops_ex = list(pd.unique(avl_data.MonitoredCallRef))
for stp in stops_ex:
    


40

In [131]:
avl_data.index.get_level_values('vehicleID').UNIQ

Index([u'MTA NYCT_4587', u'MTA NYCT_4587', u'MTA NYCT_4587', u'MTA NYCT_4587',
       u'MTA NYCT_4587', u'MTA NYCT_4587', u'MTA NYCT_4587', u'MTA NYCT_4587',
       u'MTA NYCT_4587', u'MTA NYCT_4587', 
       ...
       u'MTA NYCT_4587', u'MTA NYCT_4587', u'MTA NYCT_4587', u'MTA NYCT_4587',
       u'MTA NYCT_4587', u'MTA NYCT_4587', u'MTA NYCT_4587', u'MTA NYCT_4587',
       u'MTA NYCT_4587', u'MTA NYCT_4587'],
      dtype='object', name=u'vehicleID', length=130)